<center><h1 style="color:#0098E9;"> Topicos avanzados de Ingenieria Matematica</h1></center>
<center><h1 style="color:#0098E9;"> Tarea 2</h1></center>
<center><h3 style="color:#0098E9;"> Manuel A. Sanchez</h3></center>
<center><h3 style="color:#0098E9;"> Pontificia Universidad Catolica de Chile</h3></center>
<hr style="border:2px solid gray"> </hr>

## 4. M\'etodo de HDG usando NGSolve

Considere el problema de Poisson en el cuadrado unitario $\Omega:=(0,1)^2$

\begin{equation*}
\begin{array}{rclrl}
- \nabla\cdot (c^{-1}(x,y) \nabla u(x,y)) &=& f(x,y) & \mbox{en} & \Omega \\
g &=&g_{N}&\mbox{sobre} & \partial \Omega_{N} := \{(x,y)\in\partial \Omega: x= 1\} \\
u &=& u_{D} & \mbox{sobre} & \partial \Omega_{D} =\partial \Omega \backslash \partial \Omega_N.
\end{array}
\end{equation*}

Sea $\mathcal T_{h}$ una triangulaci\'on de $\Omega$. Entonces.

- 4.1 Utilice NGsolve para programar el problema utilizando un m\'etodo de HDG con espacios locales $V(K)  =  \mathcal P_{p}$,
$W(K) =  \mathcal P_{p}$, and $ M(\partial K) = \mathcal P_p(\partial K)$ para todo $K\in \mathcal T_{h}$ y par\'ametro de estabilizaci\'on $\tau = 1$. 
- 4.2  Testee su programa con la soluci\'on exacta $u(x,y) = \sin(x)\cos(y)$ y $c(x,y) = 1+xy$. 
- 4.3 Calcule la soluci\'on post-procesada $u_{h}^{*}$. Compare $u_h$ y $u_h^*$ por medio de un gr\'afico para una malla gruesa (aproximadamente $h=0.25$). 
- 4.4 Reporte, ya sea en una tabla o un gr\'afico log-log, los errores para las aproximaciones $u_h$, $u_h^*$ y $q_h$  y los respectivos ordenes de convergencia.  

## Respuesta.

In [13]:
#import netgen.gui
from ipywidgets.embed import embed_minimal_html, dependency_state
from ngsolve import *
from netgen.csg import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

Definimos la solución exacta

In [14]:
uexact = sin(x)*cos(y)
qexact = CoefficientFunction( ( -1/(1+x*y)*cos(x)*cos(y) , 1/(1+x*y)*sin(x)*sin(y) ) )
force = (y*cos(x)*cos(y)-x*sin(x)*sin(y))/(1+x*y)**2 + 2*cos(y)*sin(x)/(1+x*y) 
c = (1+x*y)
gN = 1/(1+x*y)*cos(x)*cos(y) #qexact #CoefficientFunction([qexact if bc=="right" else 0 for bc in mesh.GetBoundaries()])
uD = uexact

### HDG solver

In [15]:
from ngsolve.krylovspace import CGSolver

def HDGsolve(mesh,p,force,ud,gN,c):
    # Space
    Vh = VectorL2(mesh, order=p)
    Wh = L2(mesh, order=p)
    Mh = FacetFESpace(mesh, order=p, dirichlet="left|top|bottom")
    fes = FESpace([Vh,Wh,Mh])
    qh, uh, uhat = fes.TrialFunction()
    vh, wh, what = fes.TestFunction()
    # stabilization parameter
    tau = 1
    condense=True
    a = BilinearForm(fes, condense=condense)
    a += (c*qh*vh - uh*div(vh) -qh*grad(wh))*dx
    n = specialcf.normal(mesh.dim)
    qhatn = qh*n+tau*(uh-uhat)
    a += uhat*vh*n*dx(element_boundary=True)
    #a += (tau*uh*wh-tau*uhat*wh)*dx(element_boundary=True)
    a += qhatn*wh*dx(element_boundary=True)
    a += qhatn*what*dx(element_boundary=True)
    P = Preconditioner(a, "bddc")
    a.Assemble()
    
    f = LinearForm(fes)
    f += force*wh*dx
    #gN = g*n
    f += -gN*what.Trace()*ds(definedon = "right") #Condicion de Neumann
    f.Assemble()

    gf = GridFunction(fes)
    gfq, gfu, gfuhat = gf.components 
    gfuhat.Set(ud, BND)

    #Resolvemos el problema
    if condense:
        f.vec.data += a.harmonic_extension_trans * f.vec
        solvers.CG(mat = a.mat, pre=P.mat, rhs=f.vec, sol=gf.vec, initialize=False);
        gf.vec.data += a.harmonic_extension * gf.vec
        gf.vec.data += a.inner_solve * f.vec
    
    return gfu, gfq, gfuhat 

### Post-processing of $u$

In [16]:
def postprocess(mesh,p,gfu,gfq):
    fespost_u = L2(mesh, order=p+1)
    fespost_lam = L2(mesh, order=0)
    fes_post = FESpace([fespost_u, fespost_lam])

    upost, la = fes_post.TrialFunction()
    w, mu = fes_post.TestFunction()

    a = BilinearForm(fes_post)
    a += (grad(upost)*grad(w)+upost*mu)*dx
    a.Assemble()
    f = LinearForm(fes_post)
    f += (-c*gfq*grad(w)+gfu*mu)*dx
    f.Assemble()
    gfpost = GridFunction(fes_post)
    gfpost.vec.data = a.mat.Inverse() * f.vec    
    return gfpost.components[0]


### 4.3 Grafica de la solucion aproximada, triangles

In [17]:
#Iteración para plottear
p=1
mesh = Mesh(unit_square.GenerateMesh(maxh=0.5))
gfu,gfq,_ = HDGsolve(mesh,p,force,uD,gN,c)
upost = postprocess(mesh,p,gfu,gfq)

#Graficamos la solución
Draw(gfu)
#Draw(gfq)
Draw(upost, mesh, "upost")

iteration 0 error = 1.9771972946155791
iteration 1 error = 0.5746840234950987
iteration 2 error = 0.15495552251299968
iteration 3 error = 0.004328974618842344
iteration 4 error = 0.00039630641808028905
iteration 5 error = 2.431285007470811e-05
iteration 6 error = 5.892560513281211e-17


NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

## 4.2 Grafica solucion aproximada, quadrilateral

In [8]:
p=1
mesh = Mesh(unit_square.GenerateMesh(maxh=0.5, quad_dominated=True)) 
gfu,gfq,_ = HDGsolve(mesh,p,force,uD,gN,c)
#Graficamos la solución
Draw(gfu)
Draw(upost, mesh, "upost")

iteration 0 error = 1.476434778353675
iteration 1 error = 0.027391766999147818
iteration 2 error = 0.0004471828684003038
iteration 3 error = 4.496809962587334e-06
iteration 4 error = 1.1605460659536423e-07
iteration 5 error = 3.2473711990596757e-22


NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

In [9]:
#Estimación del error y del orden

def errorfun(uh,qh,uexact,duexact,upost, mesh):
    u_err = Integrate( (uh-uexact)**2 ,mesh)
    q_err = Integrate(( qh-duexact )**2, mesh)
    post_err = Integrate((upost-uexact)**2,mesh)
    return sqrt(u_err), sqrt(q_err), sqrt(post_err)


from pandas import DataFrame
def print_markdown_table(errors):
    i = 1
    h = []
    error_u = []
    order_u = []
    error_q = []
    order_q = []
    error_up = []
    order_up = []
    while i < len(errors):
        h.append(errors[i][0])
        error_u.append(errors[i][1])
        order_u.append(log(errors[i-1][1]/errors[i][1])/log(errors[i-1][0]/errors[i][0]))
        error_q.append(errors[i][2])
        order_q.append(log(errors[i-1][2]/errors[i][2])/log(errors[i-1][0]/errors[i][0]))
        error_up.append(errors[i][3])
        order_up.append( log(errors[i-1][3]/errors[i][3])/log(errors[i-1][0]/errors[i][0]))
        i += 1
    # pretty print
    df = DataFrame({"h":h,'$$\|u-u_h\|_{L^2}$$': error_u,"$$\mbox{order } u$$": order_u, "$$\|q-q_h\|_{L^2}$$": error_q, " $$\mbox{order }q$$": order_q, "$$\|u-u_h^{*}\|_{L^2}$$": error_up, " $$\mbox{order }u^*_h$$": order_up})
    display(df)


In [11]:
def convergence_test(type_ele, plot):
    p = 1
    h0 = 0.5
    output = []
    for i in range(1,6):
        if type_ele == 'quad':
            mesh = Mesh(unit_square.GenerateMesh(maxh=h0**i,quad_dominated=True))
        else:
            mesh = Mesh(unit_square.GenerateMesh(maxh=h0**i))
        uh,qh,uhat = HDGsolve( mesh, p, force, uD, gN, c)
        upost = postprocess(mesh,p,uh,qh)
        eu, eq, eup = errorfun(uh,qh,uexact,qexact, upost,mesh)
        
        output.append (( h0**i,  eu, eq, eup ))
    return output

## Convergence test triangular meshes

In [12]:
error = convergence_test('tri', plot=False)
print_markdown_table(error)

iteration 0 error = 1.9771972946155791
iteration 1 error = 0.5746840234950987
iteration 2 error = 0.15495552251299968
iteration 3 error = 0.004328974618842344
iteration 4 error = 0.00039630641808028905
iteration 5 error = 2.431285007470811e-05
iteration 6 error = 5.892560513281211e-17
iteration 0 error = 3.239386302953754
iteration 1 error = 1.285167031959893
iteration 2 error = 0.27885916843415715
iteration 3 error = 0.06480992617119319
iteration 4 error = 0.01810157378160549
iteration 5 error = 0.0024190639651282464
iteration 6 error = 0.0006130265946948363
iteration 7 error = 0.0001644843852242242
iteration 8 error = 2.8527530487543975e-05
iteration 9 error = 7.301609880668676e-06
iteration 10 error = 1.8525798304501317e-06
iteration 11 error = 5.193391850866128e-07
iteration 12 error = 9.994255369549737e-08
iteration 13 error = 2.8556554067509752e-08
iteration 14 error = 5.5172941231167485e-09
iteration 15 error = 1.2687589058419529e-09
iteration 16 error = 2.586023497383191e-10
it

,h,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$,$$\|u-u_h^{*}\|_{L^2}$$,$$\mbox{order }u^*_h$$
0,0.25000,0.001690,2.754067,0.002977,2.616827,1.441383e-04,4.292325
1,0.12500,0.000434,1.961017,0.000729,2.029703,1.792117e-05,3.007717
2,0.06250,0.000101,2.106818,0.000164,2.156645,1.813755e-06,3.304614
3,0.03125,0.000025,2.004860,0.000039,2.051027,2.227197e-07,3.025678


## Convergence test quadrilateral meshes

In [57]:
error = convergence_test('quad', plot=False)
print_markdown_table(error)

iteration 0 error = 1.476434778353675
iteration 1 error = 0.027391766999147818
iteration 2 error = 0.0004471828684003038
iteration 3 error = 4.496809962587334e-06
iteration 4 error = 1.1605460659536423e-07
iteration 5 error = 3.2473711990596757e-22
iteration 0 error = 1.9685816586440916
iteration 1 error = 0.029824068403053513
iteration 2 error = 0.0012623328481429033
iteration 3 error = 8.7264245888587e-05
iteration 4 error = 5.974812741643205e-06
iteration 5 error = 3.5864753382749496e-07
iteration 6 error = 2.341471598388426e-08
iteration 7 error = 1.3656014904590411e-09
iteration 8 error = 8.854548839676976e-11
iteration 9 error = 4.897808689772015e-12
iteration 10 error = 2.381668690355415e-13
iteration 0 error = 2.6681962060128686
iteration 1 error = 0.041947673581197935
iteration 2 error = 0.006711895844709437
iteration 3 error = 0.0010164451544004427
iteration 4 error = 0.00013243484299745527
iteration 5 error = 1.9885371874042946e-05
iteration 6 error = 3.9294559568660346e-06


,h,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$,$$\|u-u_h^{*}\|_{L^2}$$,$$\mbox{order }u^*_h$$
0,0.25000,0.002027,1.926853,0.004100,1.705737,1.956956e-04,2.734888
1,0.12500,0.000530,1.935575,0.001238,1.727632,2.895999e-05,2.756479
2,0.06250,0.000137,1.948452,0.000372,1.736388,4.224289e-06,2.777281
3,0.03125,0.000035,1.960816,0.000111,1.745048,6.099001e-07,2.792064
